In [1]:
import random
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import keras
import glob
from sklearn.preprocessing import MinMaxScaler

In [2]:
files=glob.glob("Population/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, population.shape[0]):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 2:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
all_geometry.shape

(38, 8, 3)

In [5]:
def fitness_function(pop,val):
    fitness=[0]*pop.shape[0]

    for i in range(pop.shape[0]):
        
        if (val[i][0][0][0]>=-2 and val[i][0][0][0]<=2) and (val[i][1][0][0]>=-2 and val[i][1][0][0]<=2):
            camber_fitness=2*(5-(abs(val[i][1][0][0]-val[i][0][0][0])))
            fitness[i]+=camber_fitness
        else:
            fitness[i]=0
            continue
            
        if (val[i][0][0][1]>=-1 and val[i][0][0][1]<=1) and (val[i][1][0][1]>=-1 and val[i][1][0][1]<=1):
            toe_fitness=3*(4-abs(val[i][1][0][1]-val[i][0][0][1]))
            fitness[i]+=toe_fitness
        else:
            fitness[i]=0
            continue
        
        if (val[i][0][0][2]>=4 and val[i][0][0][2]<=9) and (val[i][1][0][2]>=4 and val[i][1][0][2]<=9):
            fitness[i]+=0.5
        else:
            fitness[i]=0
            continue
            
        if (val[i][0][0][3]>=3 and val[i][0][0][3]<=8) and (val[i][1][0][3]>=3 and val[i][1][0][3]<=8):
            fitness[i]+=0.5
        else:
            fitness[i]=0
            continue

    return fitness

In [6]:
#Use tournament selection

In [7]:
def roulette_selection(population,no_parents,fitness):
    # Roulette_wheel_selection 
    previous_prob=0
    prob_sel=[x/(sum(fitness)+1e-4) for x in fitness]
    prob=[]
    parents=np.empty([no_parents,population.shape[1],population.shape[2]])
    for i in range(len(prob_sel)):
        previous_prob=previous_prob+prob_sel[i]
        prob.append(previous_prob)
    
    for i in range(no_parents):
        n=random.random()
        for j in range(len(prob)):
            if n<prob[j]:
                parents[i]=population[j]
                break
        
    return parents

In [8]:
def sus_selection(population,no_parents,fitness):
    pass

In [9]:
def crossover(parent,no_offspring,alpha):
    
    offspring=np.empty([no_offspring,parent.shape[1],parent.shape[2]])
    
    for i in range(no_offspring):
        parent1_index=i%parent.shape[0]
        parent2_index=(i+1)%parent.shape[0]
        n=random.random()
        offspring[i]=n*parent[parent1_index]+(1-n)*parent[parent2_index]
        
    return offspring

In [10]:
def mutation(offspring,mutation_rate):
    
    mutation=np.array(offspring,copy=True)
        
    for k in range(offspring.shape[0]):
        m=random.random()
        if m>mutation_rate:
            continue
            
        for j in range(offspring.shape[1]):
            for i in range(offspring.shape[2]):
                sign=random.randint(0,1)
                if sign==0:
                    mutation[k][j][i]=mutation[k][j][i]+random.random()
                else:
                    mutation[k][j][i]=mutation[k][j][i]-random.random()

    
    return mutation

In [11]:
model=keras.models.load_model('GA_ANN_normal.h5')

In [12]:
def optimization(pop,no_generation,no_parent,mutation_rate):
    bump_val=[200,-60]
    for m in range(no_generation):
        X_flat=pop.reshape((pop.shape[0], -1))
        X=[]
        X_final=[]
        bump_val=[200,-60]
        
        for i in range(X_flat.shape[0]):
            for j in range(len(bump_val)):
                X.append(list(np.append(X_flat[i],bump_val[j])))
            #X_final.append(X)
            #X=[]
            
        X_final=np.array(X)
        
        train_df=pd.DataFrame(X_final)
        scaler = MinMaxScaler(feature_range=(0,1))
        scaled_train = scaler.fit_transform(train_df)
        scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
        
        val=[]
        val_total=[]
        X = scaled_train_df[scaled_train_df.columns[:25]].values
        count=0
        for i in range(X.shape[0]):
            count+=1
            val1=model.predict(X[i].reshape(1,X[i].shape[0]))
            val.append(val1)
            if count==2:
                val_total.append(val)
                val=[]
                count=0

        pop_size=38
        no_offspring=pop_size-no_parent
        fitness=fitness_function(pop,val_total)
        parent=roulette_selection(pop,no_parent,fitness)
        offspring=crossover(parent,no_offspring,0.7)
        mutate=mutation(offspring,mutation_rate)
        pop=np.concatenate((pop,mutate),axis=0)
        
    return pop,fitness

In [13]:
pop,fitness=optimization(all_geometry,10,20,0.15)

In [14]:
index=fitness.index(max(fitness))

In [15]:
fitness[index]

22.733918264508247

In [17]:
pop[index]

array([[3976.855703724458, -180.1730421849427, 484.7899214609392],
       [4246.757845005744, -180.60425003544282, 467.03321253575245],
       [4081.6186732343476, -463.7081089321664, 255.0247544677274],
       [4063.2001696007947, -184.5262206928788, 635.4508342314095],
       [4312.179219011305, -185.2500446328654, 617.7238288772722],
       [4098.2963650176835, -454.129353937412, 419.31419455788205],
       [4031.615107687502, -489.9408601363439, 310.47904227223336],
       [4049.5346046676373, -214.28299743515922, 527.5228548590215]],
      dtype=object)

In [20]:
index

156